# Perform Classification on the ADNI dataset using NNs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
from torch.utils.data import BatchSampler, SequentialSampler, RandomSampler
import time
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from scipy.interpolate import interp1d
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV

### Read the data:

Instead of reading the whole database, we read only the data that's useful to us. That is, we read only specific columns of data, and we take only the row containing the first scan for each person. 

In "ADNI Regressional Analysis.ipynb" we have done that exactly, as well as performed linear regression transformation to the imaging data, in order to remove any age, sex, and DLICV_baseline effect. 

Furthermore, in "ADNI OPNMF.ipynb" we have performed dimensionality reduction through the OPNMF method, reducing the number of the ROIs from 145 to just 18. (Hasn't been done, so this does not apply)

Additionally, in "ADNI DeepCCA initial.ipynb" we have transformed the imaging and the genetic data using Deep Canonical Correlation Analysis to dimensionally reduced, maximally linearly correlated data. 

The data is located at "./DATA/ADNI_dataset.csv"
The transformed through LR data is located at "./DATA/Linearly_Transformed_Unique_Dataset.pkl"
The further transormed (through DCCA) data is located at "./DATA/ADNI_initial_DCCA_features.pkl"

(Need to run the RA and DCCA code if data is not found)

In [2]:
# Read the original data:
data = pd.read_csv("DATA/ADNI_dataset.csv", low_memory=False) # Need the low_memory or dtypes warning
data.replace({'Sex':{'F':1, 'M':0}}, inplace=True)

# The columns that interest us are the sex and age related, the ROIs, as well as the genetic data:
columns_of_interest = ['PTID',
                       'Date',
                       'Age', 
                       'Sex',
                       'DLICV_baseline',
                       'APOE4_Alleles',
                       'APOE_Genotype',
                       'Diagnosis_nearest_2.0']
c = list(data.columns)
MRI_columns = c[c.index("MUSE_Volume_4"):c.index("MUSE_Volume_207")+1]
genetic_columns = c[c.index("rs4575098"):c.index("rs429358")+1]

columns_of_interest += MRI_columns + genetic_columns

# Need the dropna because some first PTIDs have no MRI
data_of_interest = data[columns_of_interest].dropna(subset=['MUSE_Volume_4', 'DLICV_baseline'])


unique = data_of_interest.drop_duplicates(subset=['PTID'], keep='first')
u = unique.dropna() # only 2 values in Diagnosis_nearest_2.0' have NaN, easier to drop them:
unique = u
unique['Diagnosis_nearest_2.0'] = unique['Diagnosis_nearest_2.0'].astype('category')
unique['Diagnosis_nearest_2.0_cat'] = unique['Diagnosis_nearest_2.0'].cat.codes
print(unique.shape)
unique.head(15)

(1567, 208)


,PTID,Date,Age,Sex,DLICV_baseline,APOE4_Alleles,APOE_Genotype,Diagnosis_nearest_2.0,MUSE_Volume_4,MUSE_Volume_11,...,rs111278892,rs3752246,rs4147929,rs41289512,rs3865444,rs6024870,rs6014724,rs7274581,rs429358,Diagnosis_nearest_2.0_cat
0,002_S_0295,2006-04-18,84.742466,0,1485405.375,1.0,E3/E4,CN,1873.124153,1586.249283,...,1,1,1,0,0,0,0,0,1,0
9,002_S_0413,2006-05-02,76.283562,1,1364116.000,0.0,E3/E3,CN,2131.516933,1505.034469,...,0,1,1,0,1,0,0,0,0,0
24,002_S_0559,2006-05-23,79.223288,0,1570479.625,1.0,E3/E4,CN,2366.717680,3157.732947,...,0,0,0,0,1,0,0,0,0,0
31,002_S_0619,2006-06-01,77.447945,0,1859348.250,2.0,E4/E4,Dementia,5124.734093,2981.605944,...,0,0,0,1,1,0,0,0,2,1
36,002_S_0685,2006-07-06,89.561644,1,1372862.125,0.0,E3/E3,CN,2941.520445,1693.826402,...,1,1,1,0,0,0,0,0,0,0
45,002_S_0729,2006-07-17,65.056164,1,1166961.750,1.0,E3/E4,MCI,966.095170,1921.643449,...,0,0,0,1,1,0,0,0,1,2
64,002_S_0816,2006-08-30,70.767123,0,1444128.125,2.0,E4/E4,Dementia,1427.160586,1604.163157,...,0,0,0,0,1,0,0,0,2,1
69,002_S_0938,2006-10-05,82.167123,1,1309685.000,0.0,E3/E3,Dementia,1931.131939,1136.952611,...,0,1,1,0,1,0,0,0,0,1
74,002_S_0954,2006-10-10,69.198630,1,1075661.500,1.0,E3/E4,MCI,707.696352,2621.956978,...,2,1,1,0,1,0,0,0,1,2
81,002_S_0955,2006-10-11,78.161644,1,1363607.000,1.0,E3/E4,Dementia,2681.014413,1374.257191,...,1,0,0,0,1,0,0,0,1,1


In [3]:
# Read the data transformed through the Regressional Analysis:
lr_data = pd.read_pickle("./DATA/Linearly_Transformed_Unique_Dataset.pkl")
print(lr_data.shape)
lr_data.head(15)

(1302, 209)


,PTID,MRID,Date,Age,Sex,DLICV_baseline,APOE4_Alleles,APOE_Genotype,Diagnosis_nearest_2.0,MUSE_Volume_4,...,rs111278892,rs3752246,rs4147929,rs41289512,rs3865444,rs6024870,rs6014724,rs7274581,rs429358,Diagnosis_nearest_2.0_cat
0,002_S_0295,002_S_0295_2006-04-18,2006-04-18,84.742466,0,1485405.375,1.0,E3/E4,CN,-401.428503,...,1,1,1,0,0,0,0,0,1,0
9,002_S_0413,002_S_0413_2006-05-02,2006-05-02,76.283562,1,1364116.000,0.0,E3/E3,CN,596.355045,...,0,1,1,0,1,0,0,0,0,0
24,002_S_0559,002_S_0559_2006-05-23,2006-05-23,79.223288,0,1570479.625,1.0,E3/E4,CN,224.874560,...,0,0,0,0,1,0,0,0,0,0
31,002_S_0619,002_S_0619_2006-06-01,2006-06-01,77.447945,0,1859348.250,2.0,E4/E4,Dementia,2633.277779,...,0,0,0,1,1,0,0,0,2,1
45,002_S_0729,002_S_0729_2006-07-17,2006-07-17,65.056164,1,1166961.750,1.0,E3/E4,MCI,256.289641,...,0,0,0,1,1,0,0,0,1,2
64,002_S_0816,002_S_0816_2006-08-30,2006-08-30,70.767123,0,1444128.125,2.0,E4/E4,Dementia,-126.260419,...,0,0,0,0,1,0,0,0,2,1
69,002_S_0938,002_S_0938_2006-10-05,2006-10-05,82.167123,1,1309685.000,0.0,E3/E3,Dementia,200.102369,...,0,1,1,0,1,0,0,0,0,1
74,002_S_0954,002_S_0954_2006-10-10,2006-10-10,69.198630,1,1075661.500,1.0,E3/E4,MCI,-60.539913,...,2,1,1,0,1,0,0,0,1,2
81,002_S_0955,002_S_0955_2006-10-11,2006-10-11,78.161644,1,1363607.000,1.0,E3/E4,Dementia,1058.028132,...,1,0,0,0,1,0,0,0,1,1
84,002_S_1018,002_S_1018_2006-11-29,2006-11-29,70.658904,1,1355603.000,0.0,E3/E3,Dementia,-485.048304,...,1,1,1,0,0,0,0,0,0,1


In [4]:
# Create a new dataset and drop the imaging and genetic data:
c = list(lr_data.columns)
MRI_columns = c[c.index("MUSE_Volume_4"):c.index("MUSE_Volume_207")+1]
genetic_columns = c[c.index("rs4575098"):c.index("rs429358")+1]
columns_to_drop = MRI_columns + genetic_columns
dcca_data = lr_data.drop(labels = columns_to_drop, axis=1)

# Read the data transformed through DCCA:
with open("./DATA/ADNI_initial_DCCA_features.pkl", 'rb') as f:
    dcca_transformed_data_file = pickle.load(f)
transformed_imaging_data = dcca_transformed_data_file[0]
transformed_genetic_data = dcca_transformed_data_file[1]
print("Transformed imaging data dimensions: \n" , transformed_imaging_data.shape)
print("Transformed genetic data dimensions: \n" , transformed_genetic_data.shape)

# Embed them into the new dataset:
imaging_labels = ["imaging_component_"+str(x+1) for x in range(transformed_imaging_data.shape[1])] 
genetic_labels = ["genetic_component_"+str(x+1) for x in range(transformed_genetic_data.shape[1])] 
dcca_data[genetic_labels] = transformed_genetic_data
dcca_data[imaging_labels] = transformed_imaging_data
print("DCCA Data Dimensions: \n",dcca_data.shape)
dcca_data.head(15)

Transformed imaging data dimensions: 
 (1302, 50)
Transformed genetic data dimensions: 
 (1302, 50)
DCCA Data Dimensions: 
 (1302, 109)


/opt/homebrew/Caskroom/miniconda/base/envs/DeepCCA/lib/python3.8/site-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


,PTID,Date,Age,Sex,DLICV_baseline,APOE4_Alleles,APOE_Genotype,Diagnosis_nearest_2.0,Diagnosis_nearest_2.0_cat,genetic_component_1,...,imaging_component_41,imaging_component_42,imaging_component_43,imaging_component_44,imaging_component_45,imaging_component_46,imaging_component_47,imaging_component_48,imaging_component_49,imaging_component_50
0,002_S_0295,2006-04-18,84.742466,0,1485405.375,1.0,E3/E4,CN,0,-0.900052,...,0.265450,-0.588481,1.494160,0.464641,-0.464080,0.429959,-0.889877,2.429341,-1.496970,-4.961033
9,002_S_0413,2006-05-02,76.283562,1,1364116.000,0.0,E3/E3,CN,0,0.946432,...,-0.258067,0.501793,-1.752262,-1.133469,-1.435080,2.133784,2.246922,1.358195,-1.712850,-3.594905
24,002_S_0559,2006-05-23,79.223288,0,1570479.625,1.0,E3/E4,CN,0,0.441804,...,-0.866534,0.204544,1.261519,0.243370,-0.996479,0.513263,0.640849,3.994700,-0.652729,-5.405854
31,002_S_0619,2006-06-01,77.447945,0,1859348.250,2.0,E4/E4,Dementia,1,-1.281685,...,-2.367742,-1.426929,-1.775892,0.243927,-0.938351,-1.685831,0.941371,-3.021171,-1.171651,4.267579
45,002_S_0729,2006-07-17,65.056164,1,1166961.750,1.0,E3/E4,MCI,2,0.970438,...,-1.079121,2.071898,-2.648068,2.525044,-1.129323,1.334971,4.189767,1.001328,-2.864766,-2.184807
64,002_S_0816,2006-08-30,70.767123,0,1444128.125,2.0,E4/E4,Dementia,1,0.847462,...,-1.302565,0.370142,-1.022972,4.002663,-2.062786,-1.412294,0.224565,-1.582735,-1.250033,0.599805
69,002_S_0938,2006-10-05,82.167123,1,1309685.000,0.0,E3/E3,Dementia,1,1.102613,...,-0.744012,0.715234,-0.583494,-0.117094,-0.079651,0.871680,-0.726903,3.892656,0.357023,-4.085205
74,002_S_0954,2006-10-10,69.198630,1,1075661.500,1.0,E3/E4,MCI,2,2.527155,...,-2.024902,-0.474762,-0.873614,1.842189,-0.073196,-2.014327,2.128241,2.425868,-0.404456,-4.454329
81,002_S_0955,2006-10-11,78.161644,1,1363607.000,1.0,E3/E4,Dementia,1,0.725801,...,-0.808556,1.607881,-3.023479,1.542968,-0.623446,1.152979,4.360084,0.300277,-1.527782,-4.298677
84,002_S_1018,2006-11-29,70.658904,1,1355603.000,0.0,E3/E3,Dementia,1,0.175098,...,-1.887000,-2.865357,3.209321,1.936926,-1.822992,-0.668024,-0.510700,2.820206,1.087956,-7.384209


### Perform Classification using Neural Networks:

In [5]:
import torch.nn as nn

class NNclassifier(nn.Module):
    def __init__(self, layers, input_size, output_size):
        super().__init__()
        layers = [input_size] + layers
        # Create the NN layers, based on the sizes given:
        network_layers = []
        for i in range(len(layers) - 1):

            # if the layer isn't the last layer:
            if i != len(layers) - 2:
                size = layers[i]
                next_layer_size = layers[i+1]
                
                layer = nn.Sequential(
                    nn.Linear(size, next_layer_size),
                    nn.ReLU(),
                    nn.BatchNorm1d(next_layer_size)
                )
                
                network_layers.append(layer)
                
                
            # if the layer is the last layer:    
            else:
                size = layers[i]
                next_layer_size = output_size
                
                layer = nn.Sequential(
                    nn.Linear(size, next_layer_size),
                    nn.LogSoftmax(),   # Using logSoftmax for the loss criterion.
                    nn.BatchNorm1d(next_layer_size)
                )
                
                network_layers.append(layer)
            
            self.layers = nn.ModuleList(network_layers)    
            
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


    
def Train_NN(net, X_train, Y_train, epochs, batch_size, no_print=False):
    # Using Negative Log-Likelihood Loss Function because it's a multi-class classification problem
    criterion = nn.NLLLoss()
    # Stochastic Gradient Descent with momentum:
    optimizer = optim.SGD(net.parameters(), lr=0.002, momentum=0.9)
    
    # Get the appropriate device for calculations:
    if torch.cuda.is_available():
        device = torch.device('gpu')
    else:
        device = torch.device('cpu')
    # Get the data size:    
    data_size = X_train.shape[0]
    
    # Convert the dataset to tensor and cast them to device:
    X_train.to(device)
    X_train = X_train.type(torch.LongTensor)
    Y_train.to(device)
    Y_train = Y_train.type(torch.LongTensor)
    losses_ = []
    # Train the NN:
    for epoch in range(epochs):  # loop over the dataset multiple times
        epoch_start_time = time.time()
        # Create a set of IDs for the batch sample to train on:
        batch_idxs = list(BatchSampler(RandomSampler(range(data_size)), batch_size=batch_size, drop_last=False))
        losses = []
        # Train on each batch:
        for batch_idx in batch_idxs:
            # zero the parameter gradients
            optimizer.zero_grad()

            # get the inputs
            inputs, labels = X_train[batch_idx,:], Y_train[batch_idx]

            # forward + backward + optimize
            outputs = net(inputs.float())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        # Log the time it took for the epoch to train:
        epoch_time = time.time() - epoch_start_time
        losses_.append(losses)
        if (epoch + 1 )% 250 == 0:
            if no_print==False:
                print("Epoch ",str(epoch + 1)," training duration: ", round(epoch_time, 4), ". Loss: ", np.mean(losses))
    print('Finished Training')
    return losses_
    

def Test_NN(net, X_test, Y_test):
    correct = 0
    total = 0
    
    # Get the appropriate device for calculations:
    if torch.cuda.is_available():
        device = torch.device('gpu')
    else:
        device = torch.device('cpu')
    # Convert the dataset to tensor and cast them to device:
    X_test.to(device)
    Y_test.to(device)
    
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        inputs, labels = X_test, Y_test
        # calculate outputs by running inputs through the network
        outputs = net(inputs.float())
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

#     print('Accuracy of the network : %d %%' % (100 * correct / total))
    return (100 * correct / total)

def convert_split_dataset(X,Y):
    # Convert the numpy or pandas DF into torch tensors:
    if type(X) == pd.core.frame.DataFrame:
        X = torch.tensor(X.values.astype(np.float32))
    elif type(X) == np.ndarray:
        X = torch.from_numpy(X)
    
    if type(Y) == pd.core.series.Series:
        Y = torch.tensor(Y.values.astype(np.float32))
    elif type(Y) == np.ndarray:
        Y = torch.from_numpy(Y)

    # Split the dataset:
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.80, shuffle=False)
    return X_train, Y_train, X_test,  Y_test

In [11]:
no_print = False
layers = [512,1024,512]
epochs = 5000
batch_size = 100

print("##################################################################### Both:")
X , Y = lr_data[MRI_columns + genetic_columns], lr_data["Diagnosis_nearest_2.0_cat"]
X_train, Y_train, X_test, Y_test = convert_split_dataset(X,Y)
nn_clf_both = NNclassifier(layers,len(MRI_columns) + len(genetic_columns), 3)
losses_1 = Train_NN(nn_clf_both, X_train, Y_train, epochs, batch_size, no_print)
acc = Test_NN(nn_clf_both, X_test, Y_test)
print('Accuracy (No DCCA) : %d %%' %acc)

X , Y = dcca_data[imaging_labels + genetic_labels], dcca_data["Diagnosis_nearest_2.0_cat"]
X_train, Y_train, X_test, Y_test = convert_split_dataset(X,Y)
nn_clf_both_d = NNclassifier(layers,len(imaging_labels) + len(genetic_labels), 3)
losses_2 = Train_NN(nn_clf_both_d, X_train, Y_train, epochs, batch_size, no_print)
acc = Test_NN(nn_clf_both_d, X_test, Y_test)
print('Accuracy (DCCA) : %d %%' %acc)

print("##################################################################### Imaging only:")
X , Y = lr_data[MRI_columns], lr_data["Diagnosis_nearest_2.0_cat"]
X_train, Y_train, X_test, Y_test = convert_split_dataset(X,Y)
nn_clf_both_i = NNclassifier(layers,len(MRI_columns), 3)
losses_3 = Train_NN(nn_clf_both_i, X_train, Y_train, epochs, batch_size, no_print)
acc = Test_NN(nn_clf_both_i, X_test, Y_test)
print('Accuracy (No DCCA) : %d %%' %acc)

X , Y = dcca_data[imaging_labels], dcca_data["Diagnosis_nearest_2.0_cat"]
X_train, Y_train, X_test, Y_test = convert_split_dataset(X,Y)
nn_clf_both_i_d = NNclassifier(layers,len(imaging_labels), 3)
losses_4 = Train_NN(nn_clf_both_i_d, X_train, Y_train, epochs, batch_size, no_print)
acc = Test_NN(nn_clf_both_i_d, X_test, Y_test)
print('Accuracy (DCCA) : %d %%' %acc)

print("##################################################################### Genetic only:")
X , Y = lr_data[genetic_columns], lr_data["Diagnosis_nearest_2.0_cat"]
X_train, Y_train, X_test, Y_test = convert_split_dataset(X,Y)
nn_clf_both_g = NNclassifier(layers,len(genetic_columns), 3)
losses_5 = Train_NN(nn_clf_both_g, X_train, Y_train, epochs, batch_size, no_print)
acc = Test_NN(nn_clf_both_g, X_test, Y_test)
print('Accuracy (No DCCA) : %d %%' %acc)

X , Y = dcca_data[genetic_labels], dcca_data["Diagnosis_nearest_2.0_cat"]
X_train, Y_train, X_test, Y_test = convert_split_dataset(X,Y)
nn_clf_both_g_d = NNclassifier(layers,len(genetic_labels), 3)
losses_6 = Train_NN(nn_clf_both_g_d, X_train, Y_train, epochs, batch_size, no_print)
acc = Test_NN(nn_clf_both_g_d, X_test, Y_test)
print('Accuracy (DCCA) : %d %%' %acc)




##################################################################### Both:
Epoch  250  training duration:  0.1214 . Loss:  -52.01682524247603
Epoch  500  training duration:  0.1345 . Loss:  -106.12312802401456


KeyboardInterrupt: 

In [ ]:
flat_list_1 = [sum(sublist)/len(sublist) for sublist in losses_1]
flat_list_2 = [sum(sublist)/len(sublist) for sublist in losses_2]
flat_list_3 = [sum(sublist)/len(sublist) for sublist in losses_3]
flat_list_4 = [sum(sublist)/len(sublist) for sublist in losses_4]
flat_list_5 = [sum(sublist)/len(sublist) for sublist in losses_5]
flat_list_6 = [sum(sublist)/len(sublist) for sublist in losses_6]


plt.figure(figsize=(10,10))
x = list(range(len(flat_list_1)))
plt.plot(x, flat_list_1, label="Both, Without DCCA")
plt.plot(x, flat_list_2, label="Both, With DCCA")
plt.plot(x, flat_list_3, label="Imaging, Without DCCA")
plt.plot(x, flat_list_4, label="Imaging, With DCCA")
plt.plot(x, flat_list_5, label="Genetic, Without DCCA")
plt.plot(x, flat_list_6, label="Genetic, With DCCA")
plt.grid()
plt.legend()
plt.title("NN Training Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

## Try with balancing & scaling:

In [ ]:
# Read data:
lr_data = pd.read_pickle("./DATA/Linearly_Transformed_Unique_Dataset.pkl")

# Scale the data:
scaler = preprocessing.StandardScaler()
lr_data_scaled = scaler.fit_transform(lr_data[MRI_columns + genetic_columns])
lr_data[MRI_columns + genetic_columns] = lr_data_scaled

# Balance the data through Undersampling:
rus = RandomUnderSampler()
X = lr_data.drop('Diagnosis_nearest_2.0_cat', axis = 1)
Y = lr_data['Diagnosis_nearest_2.0_cat']
c = list(lr_data.columns)
c.remove('Diagnosis_nearest_2.0_cat')

X_res, Y_res = rus.fit_resample(X,Y)
print(Y_res.value_counts())


lr_data = pd.DataFrame(X_res,columns = c)
lr_data['Diagnosis_nearest_2.0_cat'] = Y_res
lr_data.sort_values('PTID', inplace=True,ignore_index=True)
print(lr_data.shape)
lr_data.head(10)

In [ ]:
# Scale the data:
scaler = preprocessing.StandardScaler()
dcca_data_scaled = scaler.fit_transform(dcca_data[imaging_labels + genetic_labels])
dcca_data[imaging_labels + genetic_labels] = dcca_data_scaled

# Balance the data through Undersampling - Use the same indexes as the LR Data:
dcca_data = dcca_data.loc[dcca_data['PTID'].isin(lr_data['PTID'])]

print(dcca_data.shape)
dcca_data.head(10)